In [1]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams.update({'font.size': 24})
import pathlib
import numpy as np
import pandas as pd
import xarray as xr
from scipy.stats import gaussian_kde
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import sys

In [ ]:
if 'win' in sys.platform:
    path = "E:/OneDrive/PhD/PhD/Data/Hintereisferner/Output/aws_comp/bestfiles/"
    aws_path = "E:/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/AWS_Obleitner/"
else:
    path = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Output/aws_comp/bestfiles/"
    aws_path = "/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/AWS_Obleitner/"
    
aws_lower = pd.read_csv(aws_path+"Fix_HEFlower_01102003_24102004.csv", parse_dates=True, index_col="time")
aws_lower = aws_lower.loc["2003-10-01":"2004-09-30"]
aws_upper = pd.read_csv(aws_path+"Fix_HEFupper_01102003_24102004.csv", parse_dates=True, index_col="time")
aws_upper = aws_upper.loc["2003-10-01":"2004-09-30"]
aws_upper['T'] = aws_upper['T']+273.15
aws_lower['T'] = aws_lower['T']+273.15
aws_upper['sfc'] = aws_upper['sfc'] - aws_upper['sfc'][0]
aws_lower['sfc'] =  aws_lower['sfc'] - aws_lower['sfc'][0]
print(aws_lower)

aws_lower_copy = aws_lower.copy()
aws_upper_copy = aws_upper.copy()

In [3]:
aws_lower_u2 = aws_lower[['Dir', 'U']]
aws_upper_u2 = aws_upper[['Dir', 'U']]

cos_u2_lower = pd.read_csv(path+"cosipy_at_loweraws_grid_merged-U2.csv", parse_dates=True, index_col="time")
cos_u2_lower = cos_u2_lower.loc["2003-10-01":"2004-09-30"]
if (cos_u2_lower.std(axis=1) > 1e-9).any():
    print("Warning. Windspeed not the same between runs")
cos_u2_lower = cos_u2_lower.median(axis=1)

    
cos_u2_upper = pd.read_csv(path+"cosipy_at_upperaws_grid_merged-U2.csv", parse_dates=True, index_col="time")
cos_u2_upper = cos_u2_upper.loc["2003-10-01":"2004-09-30"]
if (cos_u2_upper.std(axis=1) > 1e-9).any():
    print("Warning. Windspeed not the same between runs")
cos_u2_upper = cos_u2_upper.median(axis=1)

In [ ]:
## Short windspeed analysis.
fig, (ax1, ax2) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(15, 10),
    sharex=True,
    dpi=300
)

dir_labels = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']

def direction_to_category(deg):
    shifted_deg = (deg + 22.5) % 360
    return np.floor(shifted_deg / 45)

# Apply this function to the data
aws_lower_u2['Dir_Cat'] = direction_to_category(aws_lower_u2['Dir'])
aws_upper_u2['Dir_Cat'] = direction_to_category(aws_upper_u2['Dir'])

# Create a discrete colormap with 8 distinct colors
cmap_discrete = mcolors.ListedColormap(plt.get_cmap('Paired').colors[:8])

#color for NaN values
cmap_discrete.set_bad(color='lightgray', alpha=1.0)

ax1.plot(cos_u2_lower.index, cos_u2_lower, color='crimson', label='COSMO', zorder=6)
ax1.plot(aws_lower_u2.index, aws_lower_u2['U'], color='black', alpha=0.7, label='Lower AWS', zorder=7)
ax1.set_ylabel('Wind Speed ($m\ s^{-1}$)')
ax1.grid(linestyle='--', alpha=0.6)
ax1.legend(loc='upper left', fontsize=18).set_zorder(9)

divider = make_axes_locatable(ax1)
ax1_dir = divider.append_axes("top", size="6%", pad=0.05)
dir_data_lower = aws_lower_u2['Dir_Cat'].values[np.newaxis, :]
date_start, date_end = mdates.date2num(aws_lower_u2.index[0]), mdates.date2num(aws_lower_u2.index[-1])
norm_discrete = mcolors.BoundaryNorm(np.arange(-0.5, 8.5, 1), cmap_discrete.N)
img1 = ax1_dir.imshow(dir_data_lower, cmap=cmap_discrete, norm=norm_discrete, aspect='auto', extent=[date_start, date_end, 0, 1])

ax1_dir.yaxis.set_visible(False)
ax1_dir.xaxis.set_visible(False)

ax2.plot(cos_u2_upper.index, cos_u2_upper, color='crimson', label='COSMO', zorder=6)
ax2.plot(aws_upper_u2.index, aws_upper_u2['U'], color='black', alpha=0.7, label='Upper AWS', zorder=7)
ax2.set_ylabel('Wind Speed ($m\ s^{-1}$)')
ax2.grid(linestyle='--', alpha=0.6)
ax2.legend(loc='upper left', fontsize=18).set_zorder(9)

divider = make_axes_locatable(ax2)
ax2_dir = divider.append_axes("top", size="6%", pad=0.05)
dir_data_upper = aws_upper_u2['Dir_Cat'].values[np.newaxis, :]
img2 = ax2_dir.imshow(dir_data_upper, cmap=cmap_discrete, norm=norm_discrete, aspect='auto', extent=[date_start, date_end, 0, 1])
ax2_dir.yaxis.set_visible(False)
ax2_dir.xaxis.set_visible(False)

ax2.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 2, 3,4,5,6,7,8,9,10,11,12)))
days = mdates.DayLocator(interval=7)
ax2.xaxis.set_minor_locator(days)
ax2.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax2.xaxis.get_major_locator()))
ax2.set_xlim(pd.to_datetime("2003-10-01"), pd.to_datetime("2004-09-30"))
plt.setp(ax2.get_xticklabels(), rotation=30, ha="right")

fig.subplots_adjust(top=0.88)
cbar_ax = fig.add_axes([0.15, 0.93, 0.7, 0.03])
cbar = fig.colorbar(img1, cax=cbar_ax, orientation='horizontal')

tick_locs = np.arange(len(dir_labels))
cbar.set_ticks(tick_locs)
cbar.set_ticklabels(dir_labels)



In [ ]:
## Commented part did not account for nans!
#aws_lower = aws_lower.resample("1D").agg(T=('T', "mean"),Dir=('Dir', "mean"),U=('U', "mean"),SWIsum=('SWI', "sum"), SWImean=("SWI","mean"),SWOsum=('SWO', "sum"),SWOmean=("SWO","mean"),
#                                         LWO=('LWO','mean'),LWI=('LWI','mean'),sfc=('sfc', "mean"),RH= ('RH', "mean"),P=('P', "mean"))
aws_lower = aws_lower.resample("1D").agg(
    T=('T', lambda x: x.mean() if x.notna().all() else np.nan),
    Dir=('Dir', lambda x: x.mean() if x.notna().all() else np.nan),
    U=('U', lambda x: x.mean() if x.notna().all() else np.nan),
    SWIsum=('SWI', lambda x: x.sum(min_count=len(x)) if x.notna().all() else np.nan),
    SWImean=('SWI', lambda x: x.mean() if x.notna().all() else np.nan),
    SWOsum=('SWO', lambda x: x.sum(min_count=len(x)) if x.notna().all() else np.nan),
    SWOmean=('SWO', lambda x: x.mean() if x.notna().all() else np.nan),
    LWO=('LWO', lambda x: x.mean() if x.notna().all() else np.nan),
    LWI=('LWI', lambda x: x.mean() if x.notna().all() else np.nan),
    sfc=('sfc', lambda x: x.mean() if x.notna().all() else np.nan),
    RH=('RH', lambda x: x.mean() if x.notna().all() else np.nan),
    P=('P', lambda x: x.mean() if x.notna().all() else np.nan),
)
aws_lower['alpha'] = aws_lower['SWOsum'] / aws_lower['SWIsum']
aws_lower['sfc'] =  aws_lower['sfc'] - aws_lower['sfc'][0]

#aws_upper = aws_upper.resample("1D").agg(T=('T', "mean"),Dir=('Dir', "mean"),U=('U', "mean"),SWIsum=('SWI', "sum"), SWImean=("SWI","mean"),SWOsum=('SWO', "sum"),SWOmean=("SWO","mean"),
#                                         LWO=('LWO','mean'),LWI=('LWI','mean'),sfc=('sfc', "mean"),RH= ('RH', "mean"),P=('P', "mean"))
aws_upper = aws_upper.resample("1D").agg(
    T=('T', lambda x: x.mean() if x.notna().all() else np.nan),
    Dir=('Dir', lambda x: x.mean() if x.notna().all() else np.nan),
    U=('U', lambda x: x.mean() if x.notna().all() else np.nan),
    SWIsum=('SWI', lambda x: x.sum(min_count=len(x)) if x.notna().all() else np.nan),
    SWImean=('SWI', lambda x: x.mean() if x.notna().all() else np.nan),
    SWOsum=('SWO', lambda x: x.sum(min_count=len(x)) if x.notna().all() else np.nan),
    SWOmean=('SWO', lambda x: x.mean() if x.notna().all() else np.nan),
    LWO=('LWO', lambda x: x.mean() if x.notna().all() else np.nan),
    LWI=('LWI', lambda x: x.mean() if x.notna().all() else np.nan),
    sfc=('sfc', lambda x: x.mean() if x.notna().all() else np.nan),
    RH=('RH', lambda x: x.mean() if x.notna().all() else np.nan),
    P=('P', lambda x: x.mean() if x.notna().all() else np.nan),
)
aws_upper['alpha'] = aws_upper['SWOsum'] / aws_upper['SWIsum']
aws_upper['sfc'] = aws_upper['sfc'] - aws_upper['sfc'][0]
aws_upper

In [ ]:
vars_of_interest = ['TOTALHEIGHT','SNOWHEIGHT','MB',"SWnet", 'G','surfMB','HGT','T2','U2','RH2','LWin','MASK','RAIN', 'SNOWFALL', 'RRR', 'ME','surfM','subM', 'LWout', 'TS', 'ALBEDO','PRES']
dic_sum_lower = {}
dic_sum_upper = {}
dic_sum_lower_full = {}
dic_sum_upper_full = {}
for var in vars_of_interest:
    if var not in ["HGT", "MASK"]:
        for fp in pathlib.Path(path).glob('*{}.csv'.format(var)):
            print(fp)
            location = str(fp.stem).split('_')[2]
            if location == "upperaws":
                print(location)
                if var in ["RAIN","SNOWFALL","RRR","surfM","subM","MB"]:
                    df_og = pd.read_csv(fp, parse_dates=True, index_col="time")
                    df_upper = df_og.resample("1D").sum()
                else:
                    df_og = pd.read_csv(fp, parse_dates=True, index_col="time")
                    df_upper = df_og.resample("1D").mean()
                df_og = df_og.loc["2003-10-01":"2004-09-30"]
                df_upper = df_upper.loc["2003-10-01":"2004-09-30"]    
                if var == "TOTALHEIGHT":
                    df_og = df_og - df_og.iloc[0]
                    df_upper = df_upper - df_upper.iloc[0]
                
                summary_upper = pd.DataFrame(index=df_upper.index)
                summary_upper['ENS_MEAN'] = df_upper.mean(axis=1)
                summary_upper['ENS_MEDIAN'] = df_upper.median(axis=1)
                summary_upper['ENS_STD'] = df_upper.std(axis=1)
                summary_upper['CI_lower'] = df_upper.quantile(0.025, axis=1)
                summary_upper['CI_upper'] = df_upper.quantile(0.975, axis=1)   
                summary_upper['ENS_MIN'] = df_upper.min(axis=1)
                summary_upper['ENS_MAX'] = df_upper.max(axis=1)
                #
                summary_full_upper = pd.DataFrame(index=df_og.index)
                summary_full_upper['ENS_MEAN'] = df_og.mean(axis=1)
                summary_full_upper['ENS_MEDIAN'] = df_og.median(axis=1)
                summary_full_upper['ENS_STD'] = df_og.std(axis=1)
                summary_full_upper['CI_lower'] = df_og.quantile(0.025, axis=1)
                summary_full_upper['CI_upper'] = df_og.quantile(0.975, axis=1)   
                summary_full_upper['ENS_MIN'] = df_og.min(axis=1)
                summary_full_upper['ENS_MAX'] = df_og.max(axis=1)
                #
                dic_sum_upper[var] = summary_upper.copy()
                dic_sum_upper_full[var] = summary_full_upper.copy()
            else:
                print(location)
                if var in ["RAIN","SNOWFALL","RRR","surfM","subM","MB"]:
                    df_og = pd.read_csv(fp, parse_dates=True, index_col="time")
                    df_lower = df_og.resample("1D").sum()
                else:
                    df_og = pd.read_csv(fp, parse_dates=True, index_col="time")
                    df_lower = df_og.resample("1D").mean()
                df_og = df_og.loc["2003-10-01":"2004-09-30"]
                df_lower = df_lower.loc["2003-10-01":"2004-09-30"]
                if var == "TOTALHEIGHT":
                    df_og = df_og - df_og.iloc[0]
                    df_lower = df_lower - df_lower.iloc[0]
                    
                summary_lower = pd.DataFrame(index=df_lower.index)
                summary_lower['ENS_MEAN'] = df_lower.mean(axis=1)
                summary_lower['ENS_MEDIAN'] = df_lower.median(axis=1)
                summary_lower['ENS_STD'] = df_lower.std(axis=1)
                summary_lower['CI_lower'] = df_lower.quantile(0.025, axis=1)
                summary_lower['CI_upper'] = df_lower.quantile(0.975, axis=1)               
                summary_lower['ENS_MIN'] = df_lower.min(axis=1)
                summary_lower['ENS_MAX'] = df_lower.max(axis=1)
                #
                summary_full_lower = pd.DataFrame(index=df_og.index)
                summary_full_lower['ENS_MEAN'] = df_og.mean(axis=1)
                summary_full_lower['ENS_MEDIAN'] = df_og.median(axis=1)
                summary_full_lower['ENS_STD'] = df_og.std(axis=1)
                summary_full_lower['CI_lower'] = df_og.quantile(0.025, axis=1)
                summary_full_lower['CI_upper'] = df_og.quantile(0.975, axis=1)               
                summary_full_lower['ENS_MIN'] = df_og.min(axis=1)
                summary_full_lower['ENS_MAX'] = df_og.max(axis=1)
                #
                dic_sum_lower[var] = summary_lower.copy()
                dic_sum_lower_full[var] = summary_full_lower.copy()

In [7]:
dic_sum_lower['LWout']['ENS_MEAN'] = dic_sum_lower['LWout']['ENS_MEAN']*-1
dic_sum_upper['LWout']['ENS_MEAN'] = dic_sum_upper['LWout']['ENS_MEAN']*-1
dic_sum_lower['LWout']['ENS_MEDIAN'] = dic_sum_lower['LWout']['ENS_MEDIAN']*-1
dic_sum_upper['LWout']['ENS_MEDIAN'] = dic_sum_upper['LWout']['ENS_MEDIAN']*-1
dic_sum_lower['LWout']['ENS_MAX'] = dic_sum_lower['LWout']['ENS_MAX']*-1
dic_sum_upper['LWout']['ENS_MAX'] = dic_sum_upper['LWout']['ENS_MAX']*-1
dic_sum_lower['LWout']['ENS_MIN'] = dic_sum_lower['LWout']['ENS_MIN']*-1
dic_sum_upper['LWout']['ENS_MIN'] = dic_sum_upper['LWout']['ENS_MIN']*-1
dic_sum_lower['LWout']['CI_lower'] = dic_sum_lower['LWout']['CI_lower']*-1
dic_sum_upper['LWout']['CI_lower'] = dic_sum_upper['LWout']['CI_lower']*-1
dic_sum_lower['LWout']['CI_upper'] = dic_sum_lower['LWout']['CI_upper']*-1
dic_sum_upper['LWout']['CI_upper'] = dic_sum_upper['LWout']['CI_upper']*-1
#
dic_sum_lower_full['LWout']['ENS_MEAN'] = dic_sum_lower_full['LWout']['ENS_MEAN']*-1
dic_sum_upper_full['LWout']['ENS_MEAN'] = dic_sum_upper_full['LWout']['ENS_MEAN']*-1
dic_sum_lower_full['LWout']['ENS_MEDIAN'] = dic_sum_lower_full['LWout']['ENS_MEDIAN']*-1
dic_sum_upper_full['LWout']['ENS_MEDIAN'] = dic_sum_upper_full['LWout']['ENS_MEDIAN']*-1
dic_sum_lower_full['LWout']['ENS_MAX'] = dic_sum_lower_full['LWout']['ENS_MAX']*-1
dic_sum_upper_full['LWout']['ENS_MAX'] = dic_sum_upper_full['LWout']['ENS_MAX']*-1
dic_sum_lower_full['LWout']['ENS_MIN'] = dic_sum_lower_full['LWout']['ENS_MIN']*-1
dic_sum_upper_full['LWout']['ENS_MIN'] = dic_sum_upper_full['LWout']['ENS_MIN']*-1
dic_sum_lower_full['LWout']['CI_lower'] = dic_sum_lower_full['LWout']['CI_lower']*-1
dic_sum_upper_full['LWout']['CI_lower'] = dic_sum_upper_full['LWout']['CI_lower']*-1
dic_sum_lower_full['LWout']['CI_upper'] = dic_sum_lower_full['LWout']['CI_upper']*-1
dic_sum_upper_full['LWout']['CI_upper'] = dic_sum_upper_full['LWout']['CI_upper']*-1

In [ ]:
## Load dictionary from point LHS runs
if 'win' in sys.platform:
    filename = 'E:/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/point_scale/LHS-weighted-tenbestrmse-ens.pkl'
else:
    filename = '/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/COSIPY/point_scale/LHS-weighted-tenbestrmse-ens.pkl'

# Open the file in read-binary mode ('rb')
with open(filename, 'rb') as f:
    # Use pickle.load to load the object from the file
    loaded_results = pickle.load(f)
df_lwo = loaded_results['LWO']
df_sfc = loaded_results['SFC']
df_alb = loaded_results['ALB']
df_alb

In [ ]:
if 'win' in sys.platform:
    alb_obs = xr.open_dataset("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/HEF_processed_HRZ-30CC-filter_albedos.nc")
else:
    alb_obs = xr.open_dataset("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/HEF_processed_HRZ-30CC-filter_albedos.nc")
df_satellite_obs = alb_obs[['median_albedo']].to_dataframe()
df_satellite_obs = df_satellite_obs.loc["2003-10-01":"2004-09-30"]
df_satellite_obs

In [ ]:
#limit LWout to 316 ..
def adjust_ticks(ax, hide_x_ticks, hide_y_ticks, label):
    ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(1, 2, 3,4,5,6,7,8,9,10,11,12)))
    days = mdates.DayLocator(interval=7)
    ax.xaxis.set_minor_locator(days)
    ax.xaxis.set_major_formatter(
        mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
    ax.set_xlim(aws_upper.index[0],aws_upper.index[-1])
    
    if hide_x_ticks:
        ax.set_xticklabels("")
        ax.set_xlabel('', fontsize=22)
    else:
        ax.xaxis.set_tick_params(labelsize=18, rotation=30)
        ax.set_xlabel('Time', fontsize=22)
        
    if hide_y_ticks:
        ax.set_yticklabels("")
        ax.set_ylabel('', fontsize=22)
    else:
        ax.yaxis.set_tick_params(labelsize=18)
        ax.set_ylabel(label, fontsize=22)
        

def plot_sfc(ax, location, hide_x_ticks, hide_y_ticks, legend=False, handles_n_labels=None):
    
    if location == "upper":
        data_aws = aws_upper
        data_cos = dic_sum_upper
        df_ensemble_holder = df_upper
        label = "AWS"
    else:
        data_aws = aws_lower
        data_cos = dic_sum_lower
        df_ensemble_holder = df_lower
        label = "AWS"
        
    ax.plot(data_aws['sfc'], color="black", label=label, zorder=6)
    ax.plot(data_cos['TOTALHEIGHT']['ENS_MEDIAN'], color="red", label="Posterior Ens. Median", zorder=6)
    ax.fill_between(data_cos['TOTALHEIGHT'].index, data_cos['TOTALHEIGHT']['CI_lower'],
                    data_cos['TOTALHEIGHT']['CI_upper'], color="brown",
                    alpha=0.5, label = "Posterior 95% CI")
    if location == "upper":
        ax.plot(df_sfc['median'], color="steelblue", label="AWS-assisted Ens. Median", zorder=6)
        ax.fill_between(df_sfc['median'].index, df_sfc['lower_ci_95'],
                        df_sfc['upper_ci_95'], color="lightsteelblue",
                        alpha=0.5, label = "AWS-assisted 95% CI")
    #ax.plot(data_cos['TOTALHEIGHT']['ENS_MAX'], color="red", linestyle="dashed",linewidth=0.8, alpha=.75, label="ENS MAX", zorder=5)
    #ax.plot(data_cos['TOTALHEIGHT']['ENS_MIN'], color="red", linestyle="dashed",linewidth=0.8, alpha=.75, label="ENS MIN", zorder=5)
    #get dates where snowfall occurred
    snowfall_dates = data_cos['SNOWFALL']['ENS_MEDIAN'].loc[data_cos['SNOWFALL']['ENS_MEDIAN'] > 0]
    
    #scale values from 0.1 to 1
    snowfall_dates = (snowfall_dates - np.nanmin(snowfall_dates)) / (np.nanmax(snowfall_dates) - np.nanmin(snowfall_dates)) * (1 - 0.1) + 0.1
    for xc in snowfall_dates.index:
        ax.axvline(x=xc, linewidth=0.4, alpha=snowfall_dates[xc], color="darkblue", zorder=-1)
    '''
    ax2 = ax.twinx()
    cum_mb = (np.cumsum(data_cos['MB']['ENS_MEAN']) - np.cumsum(data_cos['MB']['ENS_MEAN'][0]))
    ax2.plot(data_cos['MB'].index, cum_mb, zorder=-1, color="darkolivegreen", alpha=1)
    ax3 = ax.twinx()
    ax3.bar(data_cos['SNOWFALL']['ENS_MEAN'].index,data_cos['SNOWFALL']['ENS_MEAN'],
            alpha=0.3,width=0.2, color="darkblue", zorder=-1)
    # right, left, top, bottom
    ax3.spines['right'].set_position(('outward', 80))
    '''
    ax.set_xlabel("Time")
                     
    ax.yaxis.label.set_color("black")
    if legend is True:
        if handles_n_labels is not None:
            handles, labels = handles_n_labels
            ax.legend(handles, labels, prop={'size': 16}, loc="lower center", ncol=1)
    ax.grid()

    adjust_ticks(ax, hide_x_ticks, hide_y_ticks, label="Height Change (m)")
    handles, labels = ax.get_legend_handles_labels()
    return (handles, labels)
     
def snowfall_lines(ax, location):
    if location == "upper":
        data_cos = dic_sum_upper
    else:
        data_cos = dic_sum_lower
        
    #get dates where snowfall occurred
    snowfall_dates = data_cos['SNOWFALL']['ENS_MEDIAN'].loc[data_cos['SNOWFALL']['ENS_MEDIAN'] > 0]
    
    min_alpha = 0.1
    max_alpha = 0.6 # <-- Capping the max alpha below 1.0 is the key fix
    
    # Scale values to the new [min_alpha, max_alpha] range
    snowfall_values = snowfall_dates.values
    scaled_alphas = (snowfall_values - np.nanmin(snowfall_values)) / (np.nanmax(snowfall_values) - np.nanmin(snowfall_values)) * (max_alpha - min_alpha) + min_alpha
    
    # Create a new series with the dates and scaled alphas
    snowfall_alphas = pd.Series(scaled_alphas, index=snowfall_dates.index)
    
    for xc, alpha_val in snowfall_alphas.items():
        ax.axvline(x=xc, linewidth=0.4, alpha=alpha_val, color="darkblue", zorder=-1)


fig, axes = plt.subplots(3,2, figsize=(18,12), dpi=300)
hand, labl = plot_sfc(ax=axes[0,0], location = "upper", hide_x_ticks=True, hide_y_ticks=False)
#axes[0,0].set_ylim(-5, 5)
axes[0,0].set_yticks(np.arange(-1, 5+1, 1))
plot_sfc(ax=axes[0,1], location = "lower", hide_x_ticks=True, hide_y_ticks=False, legend=True, handles_n_labels=(hand,labl))
axes[0,1].set_ylim(-5, 5)
axes[0,1].set_yticks(np.arange(-5, 5+1, 1))
axes[0,1].set_ylabel('', fontsize=22)
axes[0,0].set_title("Upper AWS")
axes[0,1].set_title("Lower AWS")

axes[1,0].plot(aws_upper['LWO'], color="black", label = "AWS U", zorder=6)
axes[1,0].plot(dic_sum_upper['LWout']['ENS_MEDIAN'], color="red", label="Posterior. ENS MEAN", zorder=4)
axes[1,0].fill_between(dic_sum_upper['LWout'].index, dic_sum_upper['LWout']['CI_lower'],
                dic_sum_upper['LWout']['CI_upper'], color="brown",
                alpha=0.5, label = "95% CI")
axes[1,0].plot(df_lwo['median'], color="steelblue", label="Point MEDIAN", zorder=5)
axes[1,0].fill_between(df_lwo['median'].index, df_lwo['lower_ci_95'],
                df_lwo['upper_ci_95'], color="lightsteelblue",
                alpha=0.5, label = "Point 95% CI")
#axes[1,0].plot(dic_sum_upper['LWout']['ENS_MAX'], color="red", linestyle="dashed", label="ENS MAX",linewidth=0.8, alpha=.75, zorder=5)
#axes[1,0].plot(dic_sum_upper['LWout']['ENS_MIN'], color="red", linestyle="dashed", label="ENS MIN",linewidth=0.8, alpha=.75, zorder=5)
axes[1,0].set_ylim(170, 317)
axes[1,0].set_yticks(np.arange(180, 305+25, 25))
ax1 = axes[1,0].twinx()
ax1.set_ylabel("Surf. Temp. (K)")
sigma = 5.670374419e-08

# set twin scale (convert LWO to degree celsius with emissivity of 0.99)
T_s = lambda LWo: (LWo/(sigma*0.99))**(1/4)
# get left axis limits
ymin, ymax = axes[1,0].get_ylim()
# apply function and set transformed values to right axis limits
ax1.set_ylim((T_s(ymin),T_s(ymax)))
# set an invisible artist to twin axes 
# to prevent falling back to initial values on rescale events
ax1.plot([],[])
ax1.set_yticklabels("")
ax1.set_ylabel('', fontsize=22)
snowfall_lines(axes[1,0], location="upper")

adjust_ticks(axes[1,0], hide_x_ticks=True, hide_y_ticks=False, label=r"$Q_{LWout}$"+ " (Wm$^{-2}$)")

axes[1,1].plot(aws_lower['LWO'], color="black", label = "AWS U", zorder=6)
axes[1,1].plot(dic_sum_lower['LWout']['ENS_MEDIAN'], color="red", label="ENS MEDIAN", zorder=5)
axes[1,1].fill_between(dic_sum_lower['LWout'].index, dic_sum_lower['LWout']['CI_lower'],
                dic_sum_lower['LWout']['CI_upper'], color="brown",
                alpha=0.5, label = "95% CI")
#axes[1,1].plot(dic_sum_lower['LWout']['ENS_MAX'], color="red", linestyle="dashed", label="ENS MAX",linewidth=0.8, alpha=.75, zorder=5)
#axes[1,1].plot(dic_sum_lower['LWout']['ENS_MIN'], color="red", linestyle="dashed", label="ENS MIN",linewidth=0.8, alpha=.75, zorder=5)
axes[1,1].set_ylim(170, 317)
axes[1,1].set_yticks(np.arange(180, 305+25, 25))
#(dic_sum_lower['LWout']['ENS_MEAN']*-1/(sigma*0.99))**(1/4)
ax2 = axes[1,1].twinx()
ax2.set_ylabel("Surf. Temp. (K)")

ymin, ymax = axes[1,1].get_ylim()
ax2.set_ylim((T_s(ymin),T_s(ymax)))
ax2.plot([],[])
snowfall_lines(axes[1,1], location="lower")

adjust_ticks(axes[1,1], hide_x_ticks=True, hide_y_ticks=True, label=r"$Q_{LWout}$"+ " (Wm$^{-2}$)")

axes[2,0].plot(aws_upper['alpha'], color="black", label = "AWS U", zorder=5)
axes[2,0].plot(dic_sum_upper['ALBEDO']['ENS_MEDIAN'], color="red", label="ENS MEDIAN", zorder=6)
axes[2,0].fill_between(dic_sum_upper['ALBEDO'].index, dic_sum_upper['ALBEDO']['CI_lower'],
                dic_sum_upper['ALBEDO']['CI_upper'], color="brown",
                alpha=0.5, label = "95% CI")
axes[2,0].plot(df_alb['median'], color="steelblue", label="Point MEDIAN", zorder=5)
axes[2,0].fill_between(df_alb['median'].index, df_alb['lower_ci_95'],
                df_alb['upper_ci_95'], color="lightsteelblue",
                alpha=0.5, label = "Point 95% CI")

## Markers for Landsat
axes[2,0].plot(df_satellite_obs.index, [1.03] * len(df_satellite_obs), # Plot just above the 1.0 line
               marker='|', markersize=14, ls='none', color='darkgreen', 
               label='Satellite Obs.', clip_on=False, zorder=10)
axes[2,1].plot(df_satellite_obs.index, [1.03] * len(df_satellite_obs), # Plot just above the 1.0 line
               marker='|', markersize=14, ls='none', color='darkgreen', 
               label='Satellite Obs.', clip_on=False, zorder=10)

#axes[2,0].plot(dic_sum_upper['ALBEDO']['ENS_MAX'], color="red", linestyle="dashed", linewidth=0.8, alpha=.75, label="ENS MAX", zorder=6)
#axes[2,0].plot(dic_sum_upper['ALBEDO']['ENS_MIN'], color="red", linestyle="dashed", linewidth=0.8, alpha=.75, label="ENS MIN", zorder=6)
snowfall_lines(axes[2,0], location="lower")
axes[2,0].set_ylim(0,1)
axes[2,0].set_yticks(np.arange(0,1+0.2,0.2))

adjust_ticks(axes[2,0], hide_x_ticks=False, hide_y_ticks=False, label="Albedo (-)")
axes[2,1].plot(aws_lower['alpha'], color="black", label = "AWS U", zorder=5)
axes[2,1].plot(dic_sum_lower['ALBEDO']['ENS_MEDIAN'], color="red", label="ENS MEDIAN", zorder=6)
axes[2,1].fill_between(dic_sum_lower['ALBEDO'].index, dic_sum_lower['ALBEDO']['CI_lower'],
                dic_sum_lower['ALBEDO']['CI_upper'], color="brown",
                alpha=0.5, label = "95% CI")
#axes[2,1].plot(dic_sum_lower['ALBEDO']['ENS_MAX'], color="red", linestyle="dashed", linewidth=0.8, alpha=.75, label="ENS MAX", zorder=6)
#axes[2,1].plot(dic_sum_lower['ALBEDO']['ENS_MIN'], color="red", linestyle="dashed", linewidth=0.8, alpha=.75, label="ENS MIN", zorder=6)
snowfall_lines(axes[2,1], location="lower")
adjust_ticks(axes[2,1], hide_x_ticks=False, hide_y_ticks=True, label="Albedo (-)")
axes[2,1].set_ylim(0,1)
axes[2,1].set_yticks(np.arange(0,1+0.2,0.2))

#calculate metrics like r2 and rmse
try:
    from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
except:
    from sklearn.metrics import mean_squared_error, r2_score
def calc_metrics(obs, mod):
    cleaned = obs.dropna()
    mod = mod.loc[mod.index.isin(cleaned.index)]
    try:
        rmse = root_mean_squared_error(cleaned, mod)
    except:
        rmse = mean_squared_error(cleaned, mod, squared=False)
    r2 = r2_score(cleaned, mod)
    print(rmse,r2)
    return (rmse,r2)

rmse, r2 = calc_metrics(aws_upper['sfc'], dic_sum_upper['TOTALHEIGHT']['ENS_MEDIAN'])
axes[0,0].annotate("R²: {},\nRMSE: {} m".format(round(r2, 2), round(rmse,2)), xy=(0.69, 0.78), xytext=(0.69, 0.78),
                   textcoords='axes fraction', xycoords='axes fraction', size=20)
rmse_new, r2_new = calc_metrics(aws_upper['sfc'], df_sfc['median'])
axes[0,0].annotate(
    f"R²: {r2_new:.2f},\nRMSE: {rmse_new:.2f} m",
    xy=(0.4, 0.125),  # slight vertical offset
    textcoords='axes fraction',
    xycoords='axes fraction',
    size=20,
    color='royalblue'
)
rmse, r2 = calc_metrics(aws_lower['sfc'], dic_sum_lower['TOTALHEIGHT']['ENS_MEDIAN'])
axes[0,1].annotate("R²: {},\nRMSE: {} m".format(round(r2, 2), round(rmse,2)), xy=(0.69, 0.78), xytext=(0.69, 0.78),
                   textcoords='axes fraction', xycoords='axes fraction', size=20)

aws_lower['LWO'].loc[aws_lower['LWO'] > 316] = 316
aws_upper['LWO'].loc[aws_upper['LWO'] > 316] = 316
rmse, r2 = calc_metrics(aws_upper['LWO'], dic_sum_upper['LWout']['ENS_MEDIAN'])
axes[1,0].annotate("R²: " +str(round(r2,2))+",\nRMSE: "+str(round(rmse,2))+" Wm$^{-2}$", xy=(0.57, 0.28), xytext=(0.57, 0.28),
                   textcoords='axes fraction', xycoords='axes fraction', size=20)
rmse_new, r2_new = calc_metrics(aws_upper['LWO'], df_lwo['median'])
axes[1,0].annotate(
    f"R²: {r2_new:.2f},\nRMSE: {rmse_new:.2f} Wm$^{{-2}}$",
    xy=(0.57, 0.05),  # slight vertical offset
    textcoords='axes fraction',
    xycoords='axes fraction',
    size=20,
    color='royalblue'
)
rmse, r2 = calc_metrics(aws_lower['LWO'], dic_sum_lower['LWout']['ENS_MEDIAN'])
axes[1,1].annotate("R²: " +str(round(r2,2))+",\nRMSE: "+str(round(rmse,2))+" Wm$^{-2}$", xy=(0.57, 0.28), xytext=(0.57, 0.28),
                   textcoords='axes fraction', xycoords='axes fraction', size=20)
rmse, r2 = calc_metrics(aws_upper['alpha'], dic_sum_upper['ALBEDO']['ENS_MEDIAN'])
axes[2,0].annotate("R²: {},\nRMSE: {}".format(round(r2, 2), round(rmse,2)), xy=(0.3, 0.36), xytext=(0.3, 0.36),
                   textcoords='axes fraction', xycoords='axes fraction', size=20)
rmse_new, r2_new = calc_metrics(aws_upper['alpha'], df_alb['median'])
axes[2,0].annotate(
    f"R²: {r2_new:.2f},\nRMSE: {rmse_new:.2f}",
    xy=(0.3, 0.12),  # slight vertical offset
    textcoords='axes fraction',
    xycoords='axes fraction',
    size=20,
    color='royalblue'
)
rmse, r2 = calc_metrics(aws_lower['alpha'], dic_sum_lower['ALBEDO']['ENS_MEDIAN'])
axes[2,1].annotate("R²: {},\nRMSE: {}".format(round(r2, 2), round(rmse,2)), xy=(0.3, 0.36), xytext=(0.3, 0.36),
                   textcoords='axes fraction', xycoords='axes fraction', size=20)

## add labels
fig.text(0.01, 0.95, 'a)', transform=fig.transFigure, fontsize=24)
fig.text(0.01, 0.65, 'c)', transform=fig.transFigure, fontsize=24)
fig.text(0.01, 0.36, 'e)', transform=fig.transFigure, fontsize=24)
#
fig.text(0.49, 0.95, 'b)', transform=fig.transFigure, fontsize=24)
fig.text(0.49, 0.65, 'd)', transform=fig.transFigure, fontsize=24)
fig.text(0.49, 0.36, 'f)', transform=fig.transFigure, fontsize=24)

fig.tight_layout()

if 'win' in sys.platform:
    plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/Fig08_aws_comp_lrtest.png", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/Fig08_aws_comp_lrtest.png", bbox_inches="tight")
"""
"""

In [11]:
## Define functions to calculate stats here
def calc_cdf(vals):
    x = np.sort(vals)
    y = np.arange(1, len(x)+1)/len(x)
    
    return x,y

def sc_stats(expected_vals, predicted_vals, drop_nan=False):
    if drop_nan:
        exp_nans = np.isnan(expected_vals)
        expected_vals = expected_vals[~exp_nans]
        predicted_vals = predicted_vals[~np.isnan(predicted_vals)]
        predicted_vals = predicted_vals[~exp_nans]
        expected_vals = expected_vals.loc[expected_vals.index.isin(predicted_vals.index)]
        predicted_vals = predicted_vals.loc[predicted_vals.index.isin(expected_vals.index)]
        expected_vals = expected_vals.loc[expected_vals.index.isin(predicted_vals.index)]
    xy = np.vstack([expected_vals, predicted_vals])
    z = gaussian_kde(xy)(xy)
    #sort points by density, so that densest points are plotted last
    idx = z.argsort()
    expected, predicted, z = expected_vals[idx], predicted_vals[idx], z[idx]
    return expected, predicted, z

In [12]:
## Lookup tables for bigplot .. 
aws_var_lookup = {'T2': 'T',
                 'RH2': 'RH',
                 'LWout': 'LWO',
                 'LWin': 'LWI',
                 'G': 'SWI',
                 'ALBEDO': 'alpha',
                 'TOTALHEIGHT': 'sfc',
                 'SWnet': 'SWnet',
                 'Q2': 'Q2',
                 'PRES': 'P',
                 'U2': 'U',
                 'Td': 'Td'}

minmax_lookup = {'T2': [245, 290],
                 'RH2': [0,100],
                 'LWout': [150, 350],
                 'LWin': [0, 400],
                 'G': [0, 1200],
                 'ALBEDO': [0,1],
                 'TOTALHEIGHT': [-1, 4],
                 'TOTALHEIGHT_lower': [-3.5,4],
                 'SWnet': [0, 900],
                 'Q2': [0, 14],
                 'U2': [0, 16],
                 'PRES': [670, 760],
                 'Td': [-40, 15]}

label_lookup = {'T2': "2m air temperature (K)",
               'RH2': '2m relative humidity (%)',
               'LWout': r'$Q_{LWout}$' + " (Wm$^{-2}$)",
               'LWin': r'$Q_{LWin}$' + " (Wm$^{-2}$)",
               'G': r'$Q_{SWin}$' + " (Wm$^{-2}$)",
               'ALBEDO': 'Albedo (-)',
               'TOTALHEIGHT': 'Surface Height (m)',
               'SWnet': r'$Q_{SWnet}$' + " (Wm$^{-2}$)",
               'Q2': "Specific humidity (gkg$^{-1}$)",
               'U2': "Wind speed (ms$^{-1}$)",
               'PRES': "Surface pressure (hPa)",
               'Td': "Dewpoint Temperature (°C)"}



In [13]:
plt.rcParams.update({'font.size': 22})

In [14]:
## Calculate other fields to compare, e.g. SWnet and LWnet
#aws_upper['SWnet'] = aws_upper['SWImean'] - aws_upper['SWOmean']
#aws_lower['SWnet'] = aws_lower['SWImean'] - aws_lower['SWOmean']
#
aws_upper_copy['SWnet'] = aws_upper_copy['SWI'] - aws_upper_copy['SWO']
aws_lower_copy['SWnet'] = aws_lower_copy['SWI'] - aws_lower_copy['SWO']


In [ ]:
aws_lower_copy.isnull().sum()

In [16]:
def method_EW_Sonntag(T: float) -> float:
    # Create output array with the same shape as T
    Ew = np.empty_like(T, dtype=float)
    
    # Condition for water phase
    water_mask = T >= 273.16
    # Condition for ice phase
    ice_mask = T < 273.16
    
    # Calculate over water where condition is met
    Ew[water_mask] = 6.112 * np.exp((17.67 * (T[water_mask] - 273.16)) / (T[water_mask] - 29.66))
    
    # Calculate over ice where condition is met
    Ew[ice_mask] = 6.112 * np.exp((22.46 * (T[ice_mask] - 273.16)) / (T[ice_mask] - 0.55))
    
    return Ew

def calculate_specific_humidity(relative_humidity, temperature_celsius, pressure_hpa):
    # Convert temperature from Celsius to Kelvin for your function
    temperature_kelvin = temperature_celsius + 273.15
    
    # Step 1: Calculate saturation vapor pressure (es) using YOUR function
    es = method_EW_Sonntag(temperature_kelvin)
    
    # Step 2: Calculate actual vapor pressure (e) from relative humidity
    e = (relative_humidity / 100.0) * es
    
    # Step 3: Calculate specific humidity (q)
    # This part of the formula remains the same
    q = (0.622 * e) / (pressure_hpa - (1 - 0.622) * e)
    
    return q

def calculate_dew_point(relative_humidity, temperature_celsius):
    temperature_kelvin = temperature_celsius + 273.15
    es = method_EW_Sonntag(temperature_kelvin.values)
    
    # vapor pressure (e)
    e = (relative_humidity.values / 100.0) * es
    
    # Create an output array for the results
    dew_point_celsius = np.empty_like(e, dtype=float)
    
    # ChatGPT approach following here
    e_safe = np.where(e > 0, e, 1e-10)
    val = np.log(e_safe / 6.112)

    # --- Condition 1: Dew point is above freezing (e >= 6.112 hPa) ---
    water_mask = e >= 6.112
    dew_point_celsius[water_mask] = (243.5 * val[water_mask]) / (17.67 - val[water_mask])

    # --- Condition 2: Frost point is below freezing (e < 6.112 hPa) ---
    ice_mask = e < 6.112
    dew_point_celsius[ice_mask] = (272.61 * val[ice_mask]) / (22.46 - val[ice_mask])
    
    return pd.Series(dew_point_celsius, index=temperature_celsius.index, name='dew_point')

aws_Td_upper = calculate_dew_point(aws_upper_copy['RH'], aws_upper_copy['T'] - 273.15)
aws_Td_lower = calculate_dew_point(aws_lower_copy['RH'], aws_lower_copy['T'] - 273.15)
# cos
cos_Td_upper = calculate_dew_point(dic_sum_upper_full['RH2']['ENS_MEDIAN'], dic_sum_upper_full['T2']['ENS_MEDIAN'] - 273.15)
cos_Td_lower = calculate_dew_point(dic_sum_lower_full['RH2']['ENS_MEDIAN'], dic_sum_lower_full['T2']['ENS_MEDIAN'] - 273.15)
# ci upper
cos_Td_upper_ciu = calculate_dew_point(dic_sum_upper_full['RH2']['CI_upper'], dic_sum_upper_full['T2']['CI_upper'] - 273.15)
cos_Td_lower_ciu = calculate_dew_point(dic_sum_lower_full['RH2']['CI_upper'], dic_sum_lower_full['T2']['CI_upper'] - 273.15)
# ci lower
cos_Td_upper_cil = calculate_dew_point(dic_sum_upper_full['RH2']['CI_lower'], dic_sum_upper_full['T2']['CI_lower'] - 273.15)
cos_Td_lower_cil = calculate_dew_point(dic_sum_lower_full['RH2']['CI_lower'], dic_sum_lower_full['T2']['CI_lower'] - 273.15)


# Calculate q
aws_q_upper = calculate_specific_humidity(aws_upper_copy['RH'], aws_upper_copy['T']-273.15, aws_upper_copy['P']) * 1000 #g/kg
aws_q_lower = calculate_specific_humidity(aws_lower_copy['RH'], aws_lower_copy['T']-273.15, aws_lower_copy['P']) * 1000 #g/kg
cos_q_upper = calculate_specific_humidity(dic_sum_upper_full['RH2']['ENS_MEDIAN'], dic_sum_upper_full['T2']['ENS_MEDIAN']-273.15, dic_sum_upper_full['PRES']['ENS_MEDIAN']) * 1000 #g/kg
cos_q_lower = calculate_specific_humidity(dic_sum_lower_full['RH2']['ENS_MEDIAN'], dic_sum_lower_full['T2']['ENS_MEDIAN']-273.15, dic_sum_lower_full['PRES']['ENS_MEDIAN']) * 1000 #g/kg
# ci lower
cos_q_upper_cil = calculate_specific_humidity(dic_sum_upper_full['RH2']['CI_lower'], dic_sum_upper_full['T2']['CI_lower']-273.15, dic_sum_upper_full['PRES']['CI_lower']) * 1000 #g/kg
cos_q_lower_cil = calculate_specific_humidity(dic_sum_lower_full['RH2']['CI_lower'], dic_sum_lower_full['T2']['CI_lower']-273.15, dic_sum_lower_full['PRES']['CI_lower']) * 1000 #g/kg
# ci upper
cos_q_upper_ciu = calculate_specific_humidity(dic_sum_upper_full['RH2']['CI_upper'], dic_sum_upper_full['T2']['CI_upper']-273.15, dic_sum_upper_full['PRES']['CI_upper']) * 1000 #g/kg
cos_q_lower_ciu = calculate_specific_humidity(dic_sum_lower_full['RH2']['CI_upper'], dic_sum_lower_full['T2']['CI_upper']-273.15, dic_sum_lower_full['PRES']['CI_upper']) * 1000 #g/kg
#q_g_per_kg = q_kg_per_kg * 1000 # Convert to g/kg for easier interpretation

aws_upper_copy['Q2'] = aws_q_upper
aws_lower_copy['Q2'] = aws_q_lower
dic_sum_upper_full['Q2'] = dic_sum_upper_full['RH2'].copy()
dic_sum_lower_full['Q2'] = dic_sum_upper_full['RH2'].copy()
dic_sum_upper_full['Q2']['ENS_MEDIAN'] = cos_q_upper
dic_sum_lower_full['Q2']['ENS_MEDIAN'] = cos_q_lower
dic_sum_upper_full['Q2']['CI_lower'] = cos_q_upper_cil
dic_sum_lower_full['Q2']['CI_lower'] = cos_q_lower_cil
dic_sum_upper_full['Q2']['CI_upper'] = cos_q_upper_ciu
dic_sum_lower_full['Q2']['CI_upper'] = cos_q_lower_ciu
#
aws_upper_copy['Td'] = aws_Td_upper
aws_lower_copy['Td'] = aws_Td_lower
dic_sum_upper_full['Td'] = dic_sum_upper_full['RH2'].copy()
dic_sum_lower_full['Td'] = dic_sum_upper_full['RH2'].copy()
dic_sum_upper_full['Td']['ENS_MEDIAN'] = cos_Td_upper
dic_sum_lower_full['Td']['ENS_MEDIAN'] = cos_Td_lower
dic_sum_upper_full['Td']['CI_upper'] = cos_Td_upper_ciu
dic_sum_lower_full['Td']['CI_upper'] = cos_Td_lower_ciu
dic_sum_upper_full['Td']['CI_lower'] = cos_Td_upper_cil
dic_sum_lower_full['Td']['CI_lower'] = cos_Td_lower_cil


In [ ]:
dic_sum_upper_full.keys()

In [18]:
def plot_ts(axes, var, ens_median, ci_lower, ci_upper, aws = "upper"):
    if var in ["G","SWnet","LWin"]:
        zorder_aws = 5
        zorder_ens = 6
    else:
        zorder_aws = 6
        zorder_ens = 5
        
    if aws == "upper":
        axes.plot(aws_upper_copy[aws_var_lookup[var]], color="black", label="AWS", alpha=0.9, zorder=zorder_aws)
        axes.plot(ens_median, color="red", label="COSMO", alpha=0.9, zorder=zorder_ens)
        axes.fill_between(aws_upper_copy.index, ci_lower, ci_upper, color="brown",
                    alpha=0.5)#, label = "95% CI")
        axes.grid()
    else:
        zorder_aws = 6
        axes.plot(aws_lower_copy[aws_var_lookup[var]], color="black", label="AWS", alpha=0.9, zorder=zorder_aws)
        axes.plot(ens_median, color="red", label="COSMO", alpha=0.9, zorder=zorder_ens)
        axes.fill_between(aws_lower_copy.index, ci_lower, ci_upper, color="brown",
                    alpha=0.5)#, label = "95% CI")
        axes.grid()        
    
def plot_cdf(axes, var, ens_median, ci_lower, ci_upper, aws="upper"):
    if aws == "upper":
        xaws, yaws = calc_cdf(aws_upper_copy[aws_var_lookup[var]])
        axes.plot(xaws, yaws, marker='.', color="black", linestyle='none', ms=5)
        xens, yens = calc_cdf(ens_median)
        axes.plot(xens, yens, marker='.', color="red", linestyle='none', ms=5)
        xcil, _ = calc_cdf(ci_lower)
        xciu, _ = calc_cdf(ci_upper)
        axes.fill_betweenx(yens, xcil, xciu, color='brown', alpha=0.5, label='95% CI')
    else:
        xaws, yaws = calc_cdf(aws_lower_copy[aws_var_lookup[var]])
        axes.plot(xaws, yaws, marker='.', color="black", linestyle='none', ms=5)
        xens, yens = calc_cdf(ens_median)
        axes.plot(xens, yens, marker='.', color="red", linestyle='none', ms=5)
        xcil, _ = calc_cdf(ci_lower)
        xciu, _ = calc_cdf(ci_upper)
        axes.fill_betweenx(yens, xcil, xciu, color='brown', alpha=0.5, label='95% CI')

def plot_scatter(axes, var, ens_median, ci_lower, ci_upper, aws="upper"):
    if aws == "upper":
        up_aws, up_ens, up_z = sc_stats(aws_upper_copy[aws_var_lookup[var]], ens_median, drop_nan=True)
        axes.scatter(up_aws, up_ens, c=up_z)
        #
        ci_upper = ci_upper.loc[ci_upper.index.isin(up_aws)]
        ci_lower = ci_lower.loc[ci_lower.index.isin(up_aws)]

        if var == "LWout":
            axes.errorbar(
                up_aws, up_ens,
                yerr=[up_ens - ci_upper, ci_lower - up_ens],  # Switched because I had * -1
                fmt='none', ecolor='gray', alpha=0.3, capsize=2
            )
        else:
            axes.errorbar(
                up_aws, up_ens,
                yerr=[up_ens - ci_lower, ci_upper - up_ens],  # Switched because I had * -1
                fmt='none', ecolor='gray', alpha=0.3, capsize=2
            )
    else:
        up_aws, up_ens, up_z = sc_stats(aws_lower_copy[aws_var_lookup[var]], ens_median, drop_nan=True)
        axes.scatter(up_aws, up_ens , c=up_z)
        #
        ci_upper = ci_upper.loc[ci_upper.index.isin(up_aws)]
        ci_lower = ci_lower.loc[ci_lower.index.isin(up_aws)]
        if var == "LWout":
            axes.errorbar(
                up_aws, up_ens, 
                yerr=[up_ens - ci_upper, ci_lower - up_ens],  # Switched because I had * -1
                fmt='none', ecolor='gray', alpha=0.3
            )
        else:
            axes.errorbar(
                up_aws, up_ens, 
                yerr=[up_ens - ci_lower, ci_upper - up_ens],  # Switched because I had * -1
                fmt='none', ecolor='gray', alpha=0.3
            )
    # calculate metrics and add to subplot
    mbe = np.mean(up_ens - up_aws)
    rmse = np.sqrt(mean_squared_error(up_aws, up_ens))
    r2 = r2_score(up_aws, up_ens)
    textstr = f'MBE = {mbe:.3f}\nRMSE = {rmse:.3f}\nR² = {r2:.3f}'
    axes.text(0.05, 0.95, textstr, transform=axes.transAxes,
            fontsize=20, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
          

In [19]:
def full_figure(var, lims=None):
    if var == "TOTALHEIGHT":
        ylim_min, ylim_max = minmax_lookup[var+"_lower"]
    else:
        ylim_min, ylim_max = minmax_lookup[var]

    fig = plt.figure(figsize=(24, 12), dpi=300) # Adjusted figsize for the new layout    
    # Create a 2x3 grid with custom width ratios
    # The first column will be twice as wide as the other two
    gs = gridspec.GridSpec(2, 3, figure=fig, width_ratios=[2, 1, 1])

    # Create an axes array manually to match the structure of plt.subplots
    ax = np.array([
        [fig.add_subplot(gs[0, 0]), fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2])],
        [fig.add_subplot(gs[1, 0]), fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2])]
    ])
    plot_ts(ax[0,0], var=var, ens_median=dic_sum_upper_full[var]['ENS_MEDIAN'], ci_lower=dic_sum_upper_full[var]['CI_lower'], ci_upper=dic_sum_upper_full[var]['CI_upper'], aws="upper")
    plot_ts(ax[1,0], var=var, ens_median=dic_sum_lower_full[var]['ENS_MEDIAN'], ci_lower=dic_sum_lower_full[var]['CI_lower'], ci_upper=dic_sum_lower_full[var]['CI_upper'], aws="lower")

    for xs in [ax[0,0],ax[1,0]]:
        xs.set_ylim(ylim_min, ylim_max)
        xs.set_xlabel("Date")
        xs.xaxis.set_major_locator(mdates.MonthLocator(bymonth=(10, 1, 4, 7)))
        xs.xaxis.set_minor_locator(mdates.MonthLocator())
        xs.xaxis.set_major_formatter(
            mdates.ConciseDateFormatter(xs.xaxis.get_major_locator()))
        xs.set_ylabel(label_lookup[var])
        if var == "T2":
            xs.set_yticks(np.arange(245, 290+5,5))
        elif var == "RH2":
            xs.set_yticks(np.arange(0, 100+10, 10))
        elif var == "LWout":
            xs.set_yticks(np.arange(150, 400+50, 50))
        elif var == "LWin":
            xs.set_yticks(np.arange(0, 400+50, 50))
            
    plot_cdf(ax[0,1], var=var, ens_median=dic_sum_upper_full[var]['ENS_MEDIAN'], ci_lower=dic_sum_upper_full[var]['CI_lower'], ci_upper=dic_sum_upper_full[var]['CI_upper'], aws="upper")
    plot_cdf(ax[1,1], var=var, ens_median=dic_sum_lower_full[var]['ENS_MEDIAN'], ci_lower=dic_sum_lower_full[var]['CI_lower'], ci_upper=dic_sum_lower_full[var]['CI_upper'], aws="lower")        
    for xs in [ax[0,1],ax[1,1]]:
        xs.set_xlim(ylim_min, ylim_max)
        xs.set_ylabel("Probability (%)")
        xs.set_xlabel(label_lookup[var])
        xs.set_ylim(0, 1)
        xs.set_yticks(np.arange(0,1+0.1,0.1))
        if var == "T2":
            xs.xaxis.set_minor_locator(MultipleLocator(5))
            xs.set_xticks(np.arange(245, 290+10, 10))
        elif var == "RH2":
            xs.xaxis.set_minor_locator(MultipleLocator(5))
            xs.set_xticks(np.arange(0, 100+10, 10))
        elif var == "LWout":
            xs.xaxis.set_minor_locator(MultipleLocator(10))
            xs.set_xticks(np.arange(150, 350+50, 50))    
        elif var == "LWin":
            xs.xaxis.set_minor_locator(MultipleLocator(10))
            xs.set_xticks(np.arange(0, 400+50, 50))        
        xs.grid()
        
    plot_scatter(ax[0,2], var=var, ens_median=dic_sum_upper_full[var]['ENS_MEDIAN'], ci_lower=dic_sum_upper_full[var]['CI_lower'], ci_upper=dic_sum_upper_full[var]['CI_upper'], aws="upper")
    plot_scatter(ax[1,2], var=var, ens_median=dic_sum_lower_full[var]['ENS_MEDIAN'], ci_lower=dic_sum_lower_full[var]['CI_lower'], ci_upper=dic_sum_lower_full[var]['CI_upper'], aws="lower")   
    for xs in [ax[0,2],ax[1,2]]:
        if var == "T2":
            xs.set_ylim(245, 290)
            xs.set_xlim(245, 290)
            xs.plot([245, 290], [245, 290], 'k-')
            xs.axvline(273.15, ls='-.', c='k')
            xs.axhline(273.15, ls='-.', c='k')
            xs.set_xticks(np.arange(245, 290+10, 10))
            xs.xaxis.set_minor_locator(MultipleLocator(5))
            xs.set_yticks(np.arange(245, 290+10, 10))
            xs.yaxis.set_minor_locator(MultipleLocator(5))
        elif var == "RH2":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 100+10, 10))
            xs.xaxis.set_minor_locator(MultipleLocator(5))
            xs.set_yticks(np.arange(0, 100+10, 10))
            xs.yaxis.set_minor_locator(MultipleLocator(5))
        elif var == "LWout":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(150, 350+50, 50))
            xs.xaxis.set_minor_locator(MultipleLocator(10))
            xs.set_yticks(np.arange(150, 350+50, 50))
            xs.yaxis.set_minor_locator(MultipleLocator(10))
        elif var == "LWin":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 400+50, 50))
            xs.xaxis.set_minor_locator(MultipleLocator(10))
            xs.set_yticks(np.arange(0, 400+50, 50))
            xs.yaxis.set_minor_locator(MultipleLocator(10))
            xs.xaxis.set_tick_params(rotation=45) 
        elif var == "G":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 1200+200, 200))
            xs.xaxis.set_minor_locator(MultipleLocator(50))
            xs.set_yticks(np.arange(0, 1200+200, 200))
            xs.yaxis.set_minor_locator(MultipleLocator(50))
            xs.xaxis.set_tick_params(rotation=45)  
        elif var == "SWnet":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 900+100, 100))
            xs.xaxis.set_minor_locator(MultipleLocator(50))
            xs.set_yticks(np.arange(0, 900+100, 100))
            xs.yaxis.set_minor_locator(MultipleLocator(50))
            xs.xaxis.set_tick_params(rotation=45)      
        elif var == "ALBEDO":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 1+0.2, 0.2))
            xs.set_yticks(np.arange(0, 1+0.2, 0.2))
            xs.xaxis.set_tick_params(rotation=45)
        elif var == "TOTALHEIGHT":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(-1, 4+0.5, 0.5))
            xs.set_yticks(np.arange(-1, 4+0.5, 0.5))
            xs.xaxis.set_tick_params(rotation=45)
        elif var == "U2":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 16+2, 2))
            xs.set_yticks(np.arange(0, 16+2, 2))
            xs.xaxis.set_tick_params(rotation=45)
        elif var == "Q2":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(0, 14+2, 2))
            xs.set_yticks(np.arange(0, 14+2, 2))
            xs.xaxis.set_tick_params(rotation=45)
        elif var == "Td":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(-45, 15+15, 15))
            xs.set_yticks(np.arange(-45, 15+15, 15))
            xs.xaxis.set_tick_params(rotation=45)
        elif var == "PRES":
            xs.set_ylim(ylim_min, ylim_max)
            xs.set_xlim(ylim_min, ylim_max)
            xs.plot([ylim_min, ylim_max], [ylim_min, ylim_max], 'k-')
            xs.set_xticks(np.arange(670, 760+15, 15))
            xs.set_yticks(np.arange(670, 760+15, 15))
            xs.xaxis.set_tick_params(rotation=45)
        xs.set_xlabel("AWS " + label_lookup[var].split('()')[0])
        xs.set_ylabel("COSIPY " + label_lookup[var].split('()')[0])
        xs.grid()
    plt.figtext(0.6,0.95, "Upper AWS", ha="center", va="top", fontsize=22, color="black", zorder=10)
    plt.figtext(0.6,0.45, "Lower AWS", ha="center", va="top", fontsize=22, color="black", zorder=10)
    handles, labels = ax[0,0].get_legend_handles_labels()
    print(handles)
    print(labels)
    fig.tight_layout()
    fig.legend(handles, labels, loc='upper center', ncol=5, bbox_to_anchor=(0.5, -0.01))
    if 'win' in sys.platform:
        plt.savefig("E:/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/"+"aux-fullcomp_awsvscosipy_{}.png".format(var), bbox_inches="tight")
    else:
        plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/"+"aux-fullcomp_awsvscosipy_{}.png".format(var), bbox_inches="tight")

In [ ]:
full_figure("LWout")

In [ ]:
full_figure("RH2")

In [ ]:
full_figure("T2")

In [ ]:
full_figure("LWin")

In [ ]:
full_figure("G")

In [ ]:
full_figure("SWnet")

In [ ]:
full_figure("Q2")

In [ ]:
full_figure("Td")

In [ ]:
full_figure("PRES")

In [ ]:
full_figure("U2")

In [ ]:
variables_to_plot = ['Q2', 'T2', 'PRES', 'LWin', 'G', 'U2'] #Q2 or Td
aws_var_lookup = {'T2': 'T', 'LWin': 'LWI', 'G': 'SWI', 'Q2': 'Q2', 'PRES': 'P', 'U2': 'U', 'Td': 'Td'}
unit_lookup = {'Q2': "gkg$^{-1}$", 'T2': "K", 'PRES': "hPa", "LWin": "Wm$^{-2}$", "G": "Wm$^{-2}$", "U2": "ms$^{-1}$", 'Td': "°C"}
label_lookup = {'Q2': "Q2", 'T2': "T2", 'PRES': "PRES", "LWin": r"$Q_{LWin}$", "G": r"$Q_{SWin}$", "U2": "U2", 'Td': "Td"}

axis_limits = {}
for var in variables_to_plot:
    # Combine modeled and observed data for both stations
    obs_upper = aws_upper_copy[aws_var_lookup[var]].dropna()
    mod_upper = dic_sum_upper_full[var]['ENS_MEDIAN'].dropna() # Select the correct column
    obs_lower = aws_lower_copy[aws_var_lookup[var]].dropna()
    mod_lower = dic_sum_lower_full[var]['ENS_MEDIAN'].dropna() # Select the correct column
    
    # Align data by index to ensure we only consider valid pairs
    combined_upper = pd.concat([obs_upper, mod_upper], axis=1, join='inner').values.flatten()
    combined_lower = pd.concat([obs_lower, mod_lower], axis=1, join='inner').values.flatten()
    
    all_vals = np.concatenate([combined_upper, combined_lower])
    
    # Check if all_vals is empty before calling min/max
    if all_vals.size > 0:
        var_min = all_vals.min()
        var_max = all_vals.max()
        axis_limits[var] = (var_min, var_max)
    else:
        axis_limits[var] = (0, 1) # Default fallback

# --- Helper function for hydrological year ---
def day_of_hydro_year(dt, start_month=10):
    """Calculates the day number within a hydrological year starting in a given month."""
    year = dt.year if dt.month >= start_month else dt.year - 1
    start_of_hydro_year = pd.Timestamp(year, start_month, 1)
    return (dt - start_of_hydro_year).days + 1

## Create big scatterplot
fig = plt.figure(figsize=(15, 20), dpi=300)

# Create a 5x3 grid where the 3rd row is a small spacer.
# This keeps the plot axes the same size but creates a vertical gap.
gs = gridspec.GridSpec(5, 3, figure=fig, hspace=0.20, wspace=0.25, 
                       height_ratios=[1, 1, 0.005, 1, 1],
                       top=0.97, bottom=0.15, left=0.15, right=0.97)

# Manually create the list of axes, skipping the spacer row
axes = []
# Upper station plots (rows 0 and 1 of GridSpec)
for i in range(2):
    for j in range(3):
        axes.append(fig.add_subplot(gs[i, j]))
# Lower station plots (rows 3 and 4 of GridSpec)
for i in range(3, 5):
    for j in range(3):
        axes.append(fig.add_subplot(gs[i, j]))


plot_data_sources = [
    ('Upper', dic_sum_upper_full, aws_upper_copy),
    ('Lower', dic_sum_lower_full, aws_lower_copy)
]

# This is needed because the last scatter plot is used for the colorbar
scatter = None 

plot_index = 0
i=0
letters = ["a) ","b) ","c) ","d) ","e) ","f) ","g) ","h) ","i) ","j) ","k) ","l) "]
for station_type, modeled_dic, observed_df in plot_data_sources:
    for var in variables_to_plot:
        ax = axes[plot_index]
        
        # Prepare Data for Plotting ---
        modeled_series = modeled_dic[var]['ENS_MEDIAN']
        observed_series = observed_df[aws_var_lookup[var]]
        
        aligned_mod, aligned_obs = modeled_series.align(observed_series, join='inner')
        plot_df = pd.DataFrame({'observed': aligned_obs, 'modeled': aligned_mod}).dropna()
        
        x = plot_df['observed'].values
        y = plot_df['modeled'].values
        hydro_day = plot_df.index.to_series().apply(day_of_hydro_year)
        
        if len(x) == 0: # Skip if no overlapping data
            ax.text(0.5, 0.5, f"{var}\n(No Data)", ha='center', va='center', transform=ax.transAxes)
            plot_index += 1
            continue

        # Create the Scatter Plot
        scatter = ax.scatter(x, y, c=hydro_day, s=15, cmap='viridis', alpha=0.7)
        
        # Calculate and Overlay Density Contours
        var_min, var_max = axis_limits.get(var, (np.min(x), np.max(x)))
        xx, yy = np.mgrid[var_min:var_max:100j, var_min:var_max:100j]
        positions = np.vstack([xx.ravel(), yy.ravel()])
        values = np.vstack([x, y])
        kernel = gaussian_kde(values)
        f = np.reshape(kernel(positions).T, xx.shape)
        ax.contour(xx, yy, f, colors='k', linewidths=0.4, alpha=0.5)
        
        # Formatting and Labels
        ax.text(0.05, 0.95, f"{letters[i]}{label_lookup[var]} ({unit_lookup[var]})", transform=ax.transAxes, va='top', ha='left', fontsize=20)
        i += 1
        ax.plot([var_min, var_max], [var_min, var_max], color='k', linestyle='--', lw=1)
        ax.set_xlim(var_min, var_max)
        ax.set_ylim(var_min, var_max)
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True, linestyle=':', alpha=0.6)
        
        # Statistics
        r2 = r2_score(x, y)
        rmse = np.sqrt(mean_squared_error(x, y))
        mbe = np.mean(y - x) # Mean Bias Error (Observed - Modeled) -> here flipped to mod - obs, so positive means positive bias
        stats_text = f'$R^2$ = {r2:.2f}\nMBE = {mbe:.2f}\nRMSE = {rmse:.2f}'
        ax.text(0.95, 0.05, stats_text, transform=ax.transAxes, 
                fontsize=18, verticalalignment='bottom', horizontalalignment='right',
                bbox=dict(boxstyle='round,pad=0.3', fc='white', ec='gray', alpha=0.8))

        ax.xaxis.set_major_locator(mticker.MaxNLocator(nbins=5, prune='both'))
        ax.set_yticks(ax.get_xticks())

        plot_index += 1

if scatter:
    # Position the colorbar at the very bottom
    cbar_ax = fig.add_axes([gs.left, 0.08, gs.right - gs.left, 0.015])
    cbar = fig.colorbar(scatter, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Day of Hydrological Year', fontsize=22)

fig.text((gs.left + gs.right) / 2, 0.11, 'Lower AWS', ha='center', va='center', fontsize=24)
fig.text(0.08, 0.34, 'COSMO-CLM at Lower AWS', ha='center', va='center', rotation='vertical', fontsize=24)

fig.text((gs.left + gs.right) / 2, 0.55, 'Upper AWS', ha='center', va='center', fontsize=24)
fig.text(0.08, 0.75, 'COSMO-CLM at Upper AWS', ha='center', va='center', rotation='vertical', fontsize=24)

if 'win' in sys.platform:
    plt.savefig(r"E:\OneDrive\PhD\PhD\Data\Hintereisferner\Figures\FigA02_full_scatterplot_summary_forcing.png", bbox_inches="tight")
else:
    plt.savefig("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Figures/FigA02_full_scatterplot_summary_forcing.png", bbox_inches="tight")


In [ ]:
## Load forcing and compare to SWE record from Tudes et al., 18 May 2004 reported 1463mm ± 10%
if 'win' in sys.platform:
    cosmo = pd.read_csv(r"E:\OneDrive\PhD\PhD\Data\Hintereisferner\Climate\COSMO_forcing_1999-2010_PRESintp.csv", parse_dates=True, index_col="TIMESTAMP")
else:
    cosmo = pd.read_csv("/mnt/C4AEBBABAEBB9500/OneDrive/PhD/PhD/Data/Hintereisferner/Climate/COSMO_forcing_1999-2010_PRESintp.csv", parse_dates=True, index_col="TIMESTAMP")
cosmo = cosmo.loc["2003-10-01":"2004-09-30"]
cosmo

In [ ]:
#SWE (m) = Snowfall Depth (m) × (Density of Snow / Density of Water)
mult_factor_RRR = 1
density_fresh_snow = np.maximum(109.0 + 6.0 * (cosmo['T2'] - 273.16) + 26.0 * np.sqrt(cosmo['U2']), 50.0)
snowfall = cosmo['SNOWFALL'] * mult_factor_RRR
swe_m = snowfall * (density_fresh_snow/1000.0)
swe_mm = swe_m * 1000

fig, ax = plt.subplots(1,1)
ax.plot(np.cumsum(swe_mm))
ax.plot(np.cumsum(cosmo['RRR']))
ax.errorbar(pd.to_datetime("2004-05-18"), 1463, yerr=146.3, marker="o")